# Entrega 5 - Ciencia de Dados Aplicada ao Direito II

In [7]:
# Imports
import re
import unicodedata
import pandas as pd

In [ ]:
# Carrega dados
INPUT_CSV = "data/dataset_clinica20252.csv"
df = pd.read_csv(INPUT_CSV, sep="|")

print("Dimensao:", df.shape)
print("Colunas:", list(df.columns))

Dimensao: (19800, 6)
Colunas: ['cd_causa', 'cd_atendimento', 'ds_Acao_Judicial', 'ds_fatos', 'ds_Pedidos', 'ds_Qualificacao']


In [9]:
# Funcao auxiliar para normalizar texto

def strip_accents(s: str) -> str:
    """Remove acentos de uma string para facilitar busca por regex."""
    if not isinstance(s, str): 
        return ""
    return "".join(c for c in unicodedata.normalize("NFD", s) if unicodedata.category(c) != "Mn")

In [10]:
# Sistema de pontuacao para identificar casos de credito consignado
# Usa pesos positivos (indicam credito consignado) e negativos (excluem falsos positivos)

def calcular_score_credito_consignado(texto: str) -> tuple[int, dict]:
    """
    Calcula score baseado em padroes no texto.
    
    Logica:
    - Peso +4: termos diretos (emprestimo/credito consignado)
    - Peso +3: termos tecnicos (RMC, margem consignavel, INSS+consignado)
    - Peso +2: contexto forte (aposentado+consignado, banco+emprestimo)
    - Peso +1: sinais fracos (consignado isolado, contrato financeiro)
    - Peso negativo: exclusoes (consignacao em pagamento, pensao alimenticia, tributos)
    
    Returns:
        (score_total, detalhes_dict) onde score >= 3 indica credito consignado
    """
    texto = strip_accents(texto).lower()
    detalhes = {"positivos": [], "negativos": [], "score": 0}
    score = 0
    
    # ========== SINAIS FORTES POSITIVOS ==========
    # Peso +4: termos diretos e inequivocos
    positivos_peso4 = [
        r"emprestimo\s+consignado",
        r"credito\s+consignado",
        r"cartao\s+consignado",
        r"cartao\s+de\s+credito\s+consignado",
        r"cartao\s+beneficio",
    ]
    for padrao in positivos_peso4:
        if re.search(padrao, texto):
            score += 4
            detalhes["positivos"].append(f"termo_direto: {padrao} (+4)")
    
    # Peso +3: RMC (Reserva de Margem Consignavel)
    if re.search(r"\brmc\b", texto):
        score += 3
        detalhes["positivos"].append("RMC (+3)")
    
    if re.search(r"reserva\s+de\s+margem\s+consignavel", texto):
        score += 3
        detalhes["positivos"].append("reserva margem consignavel (+3)")
    
    # Peso +3: desconto no beneficio/folha + consignado (contexto forte)
    if re.search(r"desconto.{0,30}(beneficio|folha|inss)", texto):
        if re.search(r"consignado|rmc|margem\s+consignavel", texto):
            score += 3
            detalhes["positivos"].append("desconto+consignado (+3)")
    
    # Peso +3: INSS + consignado (mas exclui contexto tributario)
    if re.search(r"\binss\b", texto):
        if not re.search(r"(contribuicao\s+previdenciaria|tributo|imposto|taxa|debito\s+fiscal|execucao\s+fiscal|divida\s+ativa)", texto):
            if re.search(r"consignado|margem\s+consignavel|rmc", texto):
                score += 3
                detalhes["positivos"].append("INSS+consignado (+3)")
    
    # Peso +3: portabilidade/refinanciamento + consignado
    if re.search(r"portabilidade|refinanciam", texto):
        if re.search(r"consignado|rmc", texto):
            score += 3
            detalhes["positivos"].append("portabilidade/refinanc+consignado (+3)")
    
    # ========== SINAIS MEDIOS POSITIVOS ==========
    # Peso +2: aposentado/pensionista + consignado
    if re.search(r"aposentad|pensionista", texto):
        if re.search(r"consignado|rmc", texto):
            score += 2
            detalhes["positivos"].append("aposentado/pensionista+consignado (+2)")
    
    # Peso +2: margem consignavel isolada
    if re.search(r"margem\s+consignavel", texto):
        score += 2
        detalhes["positivos"].append("margem_consignavel (+2)")
    
    # Peso +2: desconto indevido + contexto previdenciario
    if re.search(r"desconto.{0,40}indevido", texto):
        if re.search(r"beneficio|inss|folha", texto):
            score += 2
            detalhes["positivos"].append("desconto_indevido+contexto (+2)")
    
    # Peso +2: banco + emprestimo + consignado (contexto financeiro completo)
    if re.search(r"\b(banco|instituicao\s+financeira)\b", texto):
        if re.search(r"emprestimo|credito", texto) and re.search(r"consignado", texto):
            score += 2
            detalhes["positivos"].append("banco+emprestimo+consignado (+2)")
    
    # ========== SINAIS FRACOS POSITIVOS ==========
    # Peso +1: palavra "consignado" isolada (pode ser qualquer contexto)
    if re.search(r"\bconsignado\b", texto):
        score += 1
        detalhes["positivos"].append("consignado_isolado (+1)")
    
    # Peso +1: contrato de emprestimo/financiamento/credito (contexto generico)
    if re.search(r"contrato.{0,30}(emprestimo|financiamento|credito)", texto):
        score += 1
        detalhes["positivos"].append("contrato_financeiro (+1)")
    
    # ========== EXCLUSOES (FALSOS POSITIVOS) ==========
    # Peso -5: consignacao em pagamento (acao judicial especifica, nao e credito)
    if re.search(r"consignacao\s+em\s+pagamento", texto):
        score -= 5
        detalhes["negativos"].append("consignacao_em_pagamento (-5)")
    
    if re.search(r"acao\s+de\s+consignacao", texto):
        score -= 5
        detalhes["negativos"].append("acao_de_consignacao (-5)")
    
    # Peso -4: deposito em consignacao (contexto judicial, nao e credito)
    if re.search(r"deposito\s+(em|judicial)\s+consignacao", texto):
        score -= 4
        detalhes["negativos"].append("deposito_consignacao (-4)")
    
    # Peso -4: mercadoria em consignacao (contexto comercial)
    if re.search(r"mercadoria.{0,20}consignacao", texto):
        score -= 4
        detalhes["negativos"].append("mercadoria_consignacao (-4)")
    
    # Peso -4: consignado alimentar (pensao alimenticia, nao e emprestimo)
    if re.search(r"consignado\s+alimentar", texto):
        score -= 4
        detalhes["negativos"].append("consignado_alimentar (-4)")
    
    if re.search(r"pensao\s+alimenticia", texto):
        if re.search(r"consignado|desconto", texto):
            score -= 3
            detalhes["negativos"].append("pensao_alimenticia (-3)")
    
    # Peso -3: consignacao de alugueis
    if re.search(r"aluguel|locacao", texto):
        if re.search(r"consignacao", texto):
            score -= 3
            detalhes["negativos"].append("consignacao_aluguel (-3)")
    
    # Peso -2: contexto trabalhista puro (sem mencao a INSS/aposentadoria)
    if re.search(r"(trabalhista|reclamacao\s+trabalhista|tst|clt)", texto):
        if not re.search(r"inss|aposentad|pensionista|beneficio\s+previdenciario", texto):
            if re.search(r"consignado|consignacao", texto):
                score -= 2
                detalhes["negativos"].append("contexto_trabalhista_puro (-2)")
    
    # Peso -4: contexto tributario/fiscal com INSS (execucao fiscal, divida ativa)
    if re.search(r"(execucao\s+fiscal|divida\s+ativa|debito\s+tributario|cobranca\s+de\s+tributo)", texto):
        if re.search(r"inss", texto):
            score -= 4
            detalhes["negativos"].append("contexto_tributario_INSS (-4)")
    
    # Peso -3: revisao de beneficio SEM mencao a emprestimo/credito consignado
    if re.search(r"(revisao\s+de\s+beneficio|concessao\s+de\s+beneficio|aposentadoria\s+por\s+idade)", texto):
        if re.search(r"\binss\b", texto):
            if not re.search(r"(emprestimo|credito|desconto|consignado|rmc)", texto):
                score -= 3
                detalhes["negativos"].append("revisao_previdenciaria_pura (-3)")
    
    # Peso -2: outros beneficios previdenciarios (auxilio-doenca, salario-maternidade, BPC/LOAS)
    if re.search(r"(auxilio.doenca|auxilio.acidente|salario.maternidade|bpc|loas)", texto):
        score -= 2
        detalhes["negativos"].append("beneficio_previdenciario_outro (-2)")
    
    detalhes["score"] = score
    return score, detalhes

In [13]:
def filtra_credito_consignado_robusto_v2(df: pd.DataFrame, score_minimo: int = 3, 
                                         exigir_sinal_forte: bool = True) -> pd.DataFrame:
    """
    Versao melhorada que EXIGE pelo menos um sinal forte para evitar falsos positivos.
    
    Fluxo:
    1. Para cada linha, concatena todas as colunas de texto
    2. Calcula score usando calcular_score_credito_consignado()
    3. Verifica se score >= score_minimo E tem pelo menos um sinal forte (peso 3 ou 4)
    4. Salva apenas cd_atendimento em output.xlsx
    
    Args:
        df: DataFrame com colunas ds_Acao_Judicial, ds_fatos, ds_Pedidos, ds_Qualificacao
        score_minimo: threshold para considerar como credito consignado (default: 3)
        exigir_sinal_forte: se True, exige pelo menos um sinal forte (peso 3 ou 4)
    
    Returns:
        DataFrame com cd_atendimento, score e detalhes dos casos filtrados
    """
    text_cols = ["ds_Acao_Judicial", "ds_fatos", "ds_Pedidos", "ds_Qualificacao"]
    present = [c for c in text_cols if c in df.columns]
    
    resultados = []
    
    for idx, row in df.iterrows():
        # Concatena textos
        texto_completo = " ".join(str(row[c]) for c in present if pd.notna(row[c]))
        
        # Calcula score
        score, detalhes = calcular_score_credito_consignado(texto_completo)
        
        # Verifica threshold de score
        if score >= score_minimo:
            # Verifica se tem sinal forte (caso exigido)
            if exigir_sinal_forte:
                positivos_str = "; ".join(detalhes["positivos"])
                sinais_fortes = [
                    "termo_direto",
                    "RMC",
                    "reserva margem consignavel",
                    "desconto+consignado",
                    "INSS+consignado",
                    "portabilidade/refinanc+consignado"
                ]
                tem_sinal_forte = any(sinal in positivos_str for sinal in sinais_fortes)
                
                # Se nao tem sinal forte, pula este caso
                if not tem_sinal_forte:
                    continue
            
            # Adiciona aos resultados
            resultados.append({
                "cd_atendimento": str(row["cd_atendimento"]),
                "score": score,
                "positivos": "; ".join(detalhes["positivos"]),
                "negativos": "; ".join(detalhes["negativos"]) if detalhes["negativos"] else "",
            })
    
    resultado_df = pd.DataFrame(resultados)
    
    # Salva arquivo Excel
    if len(resultado_df) > 0:
        resultado_df[["cd_atendimento"]].to_excel("output/output.xlsx", index=False)
        print(f"\n{len(resultado_df)} casos identificados e salvos em output.xlsx")
    else:
        print(f"\nNenhum caso identificado com score >= {score_minimo}")
    
    return resultado_df

In [12]:
# Executa o filtro
print("Analisando casos para identificar credito consignado...")
print("=" * 80)

resultado = filtra_credito_consignado_robusto_v2(df, score_minimo=3, exigir_sinal_forte=True)

print("\n" + "=" * 80)
print(f"ESTATISTICAS:")
print(f"  Total de casos no dataset: {len(df)}")
print(f"  Casos identificados como credito consignado: {len(resultado)}")
print(f"  Percentual: {len(resultado)/len(df)*100:.2f}%")

if len(resultado) > 0:
    print(f"\nDistribuicao de scores (top 10):")
    print(resultado['score'].value_counts().sort_index(ascending=False).head(10))

resultado.head(10)

Analisando casos para identificar credito consignado...

8556 casos identificados e salvos em output.xlsx

ESTATISTICAS:
  Total de casos no dataset: 19800
  Casos identificados como credito consignado: 8556
  Percentual: 43.21%

Distribuicao de scores (top 10):
score
38     2
37     2
35     2
34     4
33     3
32     2
31     6
30     3
29    16
28     6
Name: count, dtype: int64


,cd_atendimento,score,positivos,negativos
0,0800194-50.2025.8.10.0126,10,desconto+consignado (+3); INSS+consignado (+3)...,
1,1009294-25.2025.8.26.0506,7,desconto+consignado (+3); aposentado/pensionis...,
2,1017110-70.2025.8.26.0405,8,desconto+consignado (+3); aposentado/pensionis...,
3,0002561-03.2025.8.16.0130,18,termo_direto: emprestimo\s+consignado (+4); de...,
4,0800681-57.2025.8.18.0074,10,INSS+consignado (+3); aposentado/pensionista+c...,
5,0801193-31.2025.8.18.0077,12,termo_direto: cartao\s+consignado (+4); INSS+c...,
6,0801915-71.2024.8.10.0126,19,termo_direto: credito\s+consignado (+4); termo...,
7,0141882-95.2025.8.04.1000,10,termo_direto: emprestimo\s+consignado (+4); ap...,
8,0702310-89.2025.8.02.0046,7,desconto+consignado (+3); INSS+consignado (+3)...,beneficio_previdenciario_outro (-2)
9,0800458-85.2025.8.18.0048,6,desconto+consignado (+3); aposentado/pensionis...,
